# Reslice and mask

Use SPM reslice function to resample the masks to each functional space, and then use fsl's ApplyMask to extract the data of interest

In [1]:
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
import nipype.interfaces.fsl.maths as fsl
from nipype.interfaces import spm as spm
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html

matlab_cmd = '/opt/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
msks = pe.Workflow(name="msks") # workflow to run the analysis

# Grab and save data

In [3]:
# THIS CODE DEFINITELY WORKS
dg = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                         outfields=['masks','con', 'CNR', 'RMS']), name='data-grabber')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'masks': [['sub']],
                           'con':[['sub', 'TR']],
                           'CNR': [['sub', 'TR']],
                           'RMS': [['sub', 'TR']]}
dg.inputs.field_template = {'masks': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/roi-masks/sub-%s/*/*.nii.gz',
                            'con': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/TR%s/FLGLM/con_0001.nii.gz',
                            'CNR': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/TR%s/FLGLM/CNR_*.nii.gz',
                            'RMS': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/TR%s/FLGLM/RMS_CNR_*.nii.gz'}

In [4]:
# # debugging
# dg.inputs.sub = '05'
# dg.inputs.TR = '1920'
# res = dg.run()
# res.outputs

In [5]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath)
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0], [t for t in l if re.search('TR([0-9])', t)][0]
    name = '/'.join(name)
    name
    return name

In [6]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_gunzipmask([0-9]*)', ''),
                 ('_mask-CNR([0-9]*)', ''),
                 ('_mask-RMS([0-9]*)', '')]
ds.inputs.regexp_substitutions = substitutions

In [7]:
# THIS WORKS
datinf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                 name='dat-info')
datinf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
#datinf.iterables = [('sub', ['05']), ('TR', ['1920'])]
datinf.iterables 

[('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]

# Gunzip node

In [8]:
gunzipmany = pe.MapNode(Gunzip(), name='gunzipmask', iterfield=['in_file'])

In [9]:
gunzipcon = pe.Node(Gunzip(), name='gunzipcon')

# Reslice node

In [10]:
reslice = pe.MapNode(spm.Reslice(), iterfield=['in_file'], name='reslice')

# Gzip interface 
Required to get data from .nii (spm) to .nii.gz (fsl)

In [11]:
from nipype.interfaces.base import (
    TraitedSpec,
    CommandLineInputSpec,
    CommandLine,
    File
)
import os

class GZipInputSpec(CommandLineInputSpec):
    in_file = File(desc="File", exists=True, mandatory=True, argstr="%s")

class GZipOutputSpec(TraitedSpec):
    out_file = File(desc = "Zip file", exists = True)

class GZipTask(CommandLine):
    input_spec = GZipInputSpec
    output_spec = GZipOutputSpec
    _cmd = 'gzip'

    def _list_outputs(self):
            outputs = self.output_spec().get()
            outputs['out_file'] = os.path.abspath(self.inputs.in_file + ".gz")
            return outputs
        
gzip = pe.MapNode(GZipTask(), iterfield='in_file', name='gzip')
        
# if __name__ == '__main__':

#     zipper = GZipTask(input_file='an_existing_file')
#     print zipper.cmdline 
#     zipper.run()

Functions to concatenate the filelists into complementary lists:

In [12]:
def cnrCat(fullFileList, nMasks=9):
    return fullFileList*nMasks

In [13]:
#cnrCat(res.outputs.CNR)

In [14]:
def MskCat(fullFileList, nCon=7):
    import numpy as np
    return np.ndarray.tolist(np.repeat(fullFileList, nCon))

In [15]:
#MskCat(res.outputs.masks)

# define out_file names for the fsl masking functions

In [16]:
# first get a list of all the confounds files for the single echo TRs to which I will apply the printCensored and printMotion functions below
CNR_names = ['CNR_0001', 'CNR_0002', 'CNR_0003', 'CNR_0005', 'CNR_0006', 'CNR_0007', 'CNR_0009']
RMS_names = ['RMS_CNR_0001', 'RMS_CNR_0002', 'RMS_CNR_0003', 'RMS_CNR_0005', 'RMS_CNR_0006', 'RMS_CNR_0007', 'RMS_CNR_0009']
masks = ['FEF', 'IPS', 'LOC', 'CN', 'GPe', 'GPi', 'Put', 'STN', 'VS']
CNR_fnames = [i+'_'+n+'.nii.gz' for i in masks for n in CNR_names]
RMS_fnames = [i+'_'+n+'.nii.gz' for i in masks for n in RMS_names]
CNR_fnames

['FEF_CNR_0001.nii.gz',
 'FEF_CNR_0002.nii.gz',
 'FEF_CNR_0003.nii.gz',
 'FEF_CNR_0005.nii.gz',
 'FEF_CNR_0006.nii.gz',
 'FEF_CNR_0007.nii.gz',
 'FEF_CNR_0009.nii.gz',
 'IPS_CNR_0001.nii.gz',
 'IPS_CNR_0002.nii.gz',
 'IPS_CNR_0003.nii.gz',
 'IPS_CNR_0005.nii.gz',
 'IPS_CNR_0006.nii.gz',
 'IPS_CNR_0007.nii.gz',
 'IPS_CNR_0009.nii.gz',
 'LOC_CNR_0001.nii.gz',
 'LOC_CNR_0002.nii.gz',
 'LOC_CNR_0003.nii.gz',
 'LOC_CNR_0005.nii.gz',
 'LOC_CNR_0006.nii.gz',
 'LOC_CNR_0007.nii.gz',
 'LOC_CNR_0009.nii.gz',
 'CN_CNR_0001.nii.gz',
 'CN_CNR_0002.nii.gz',
 'CN_CNR_0003.nii.gz',
 'CN_CNR_0005.nii.gz',
 'CN_CNR_0006.nii.gz',
 'CN_CNR_0007.nii.gz',
 'CN_CNR_0009.nii.gz',
 'GPe_CNR_0001.nii.gz',
 'GPe_CNR_0002.nii.gz',
 'GPe_CNR_0003.nii.gz',
 'GPe_CNR_0005.nii.gz',
 'GPe_CNR_0006.nii.gz',
 'GPe_CNR_0007.nii.gz',
 'GPe_CNR_0009.nii.gz',
 'GPi_CNR_0001.nii.gz',
 'GPi_CNR_0002.nii.gz',
 'GPi_CNR_0003.nii.gz',
 'GPi_CNR_0005.nii.gz',
 'GPi_CNR_0006.nii.gz',
 'GPi_CNR_0007.nii.gz',
 'GPi_CNR_0009.nii.gz',

# Use FSL to mask the CNR and the RMS images

In [17]:
maskCNR = pe.MapNode(fsl.ApplyMask(), iterfield = ['in_file', 'mask_file', 'out_file'], name='mask-CNR')
maskCNR.inputs.out_file = CNR_fnames

In [18]:
maskRMS = pe.MapNode(fsl.ApplyMask(), iterfield = ['in_file', 'mask_file', 'out_file'], name='mask-RMS')
maskRMS.inputs.out_file = RMS_fnames

In [19]:
msks.connect([(datinf, dg, [('sub', 'sub')]),
              (datinf, dg, [('TR', 'TR')]),
              (dg, ds, [(('con', printSubPath), 'container')]),
              (dg, gunzipmany, [('masks', 'in_file')]),
              (dg, gunzipcon, [('con', 'in_file')]),
              (gunzipmany, reslice, [('out_file', 'in_file')]),
              (gunzipcon, reslice, [('out_file', 'space_defining')]),
              (reslice, gzip, [('out_file', 'in_file')]),
              (gzip, ds, [('out_file', 'MASKS.@RESLICE')]),
              (gzip, maskCNR, [(('out_file', MskCat), 'mask_file')]),
              (dg, maskCNR, [(('CNR', cnrCat), 'in_file')]),
              (maskCNR, ds, [('out_file', 'MASKS.CNR')]),
              (gzip, maskRMS, [(('out_file', MskCat), 'mask_file')]),
              (dg, maskRMS, [(('RMS', cnrCat), 'in_file')]),
              (maskRMS, ds, [('out_file', 'MASKS.RMS')])
])
res = msks.run()

# Reslice node

200914-19:31:26,106 nipype.workflow INFO:
	 Workflow msks settings: ['check', 'execution', 'logging', 'monitoring']
200914-19:31:26,251 nipype.workflow INFO:
	 Running serially.
200914-19:31:26,253 nipype.workflow INFO:
	 [Node] Setting-up "msks.data-grabber" in "/tmp/tmp2chmu2jx/msks/_TR_1920_sub_05/data-grabber".
200914-19:31:26,259 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
200914-19:31:26,303 nipype.workflow INFO:
	 [Node] Finished "msks.data-grabber".
200914-19:31:26,306 nipype.workflow INFO:
	 [Node] Setting-up "msks.gunzipcon" in "/tmp/tmp4_axbxoi/msks/_TR_1920_sub_05/gunzipcon".
200914-19:31:26,315 nipype.workflow INFO:
	 [Node] Running "gunzipcon" ("nipype.algorithms.misc.Gunzip")
200914-19:31:26,342 nipype.workflow INFO:
	 [Node] Finished "msks.gunzipcon".
200914-19:31:26,343 nipype.workflow INFO:
	 [Node] Setting-up "msks.gunzipmask" in "/tmp/tmp2arwl9cp/msks/_TR_1920_sub_05/gunzipmask".
200914-19:31:26,349 nipype.workflow INFO

200914-19:32:00,454 nipype.workflow INFO:
	 [Node] Setting-up "_gzip2" in "/tmp/tmpnsub8eg1/msks/_TR_1920_sub_05/gzip/mapflow/_gzip2".
200914-19:32:00,458 nipype.workflow INFO:
	 [Node] Running "_gzip2" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmp2arwl9cp/msks/_TR_1920_sub_05/gunzipmask/mapflow/_gunzipmask2/rLOC_trans.nii
200914-19:32:00,570 nipype.workflow INFO:
	 [Node] Finished "_gzip2".
200914-19:32:00,572 nipype.workflow INFO:
	 [Node] Setting-up "_gzip3" in "/tmp/tmpnsub8eg1/msks/_TR_1920_sub_05/gzip/mapflow/_gzip3".
200914-19:32:00,576 nipype.workflow INFO:
	 [Node] Running "_gzip3" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmp2arwl9cp/msks/_TR_1920_sub_05/gunzipmask/mapflow/_gunzipmask3/rCN_trans.nii
200914-19:32:00,690 nipype.workflow INFO:
	 [Node] Finished "_gzip3".
200914-19:32:00,692 nipype.workflow INFO:
	 [Node] Setting-up "_gzip4" in "/tmp/tmpnsub8eg1/msks/_TR_1920_sub_05/gzip/mapflow/_gzip4".
200914-19:32:00,69

200914-19:32:03,66 nipype.workflow INFO:
	 [Node] Finished "_mask-RMS8".
200914-19:32:03,69 nipype.workflow INFO:
	 [Node] Setting-up "_mask-RMS9" in "/tmp/tmp0qoemq2m/msks/_TR_1920_sub_05/mask-RMS/mapflow/_mask-RMS9".
200914-19:32:03,73 nipype.workflow INFO:
	 [Node] Running "_mask-RMS9" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/RMS_CNR_0003.nii.gz -mas /tmp/tmp2arwl9cp/msks/_TR_1920_sub_05/gunzipmask/mapflow/_gunzipmask1/rIPS_trans.nii.gz IPS_RMS_CNR_0003.nii.gz
200914-19:32:03,250 nipype.workflow INFO:
	 [Node] Finished "_mask-RMS9".
200914-19:32:03,252 nipype.workflow INFO:
	 [Node] Setting-up "_mask-RMS10" in "/tmp/tmp0qoemq2m/msks/_TR_1920_sub_05/mask-RMS/mapflow/_mask-RMS10".
200914-19:32:03,258 nipype.workflow INFO:
	 [Node] Running "_mask-RMS10" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/d

200914-19:32:05,635 nipype.workflow INFO:
	 [Node] Finished "_mask-RMS22".
200914-19:32:05,637 nipype.workflow INFO:
	 [Node] Setting-up "_mask-RMS23" in "/tmp/tmp0qoemq2m/msks/_TR_1920_sub_05/mask-RMS/mapflow/_mask-RMS23".
200914-19:32:05,643 nipype.workflow INFO:
	 [Node] Running "_mask-RMS23" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/RMS_CNR_0003.nii.gz -mas /tmp/tmp2arwl9cp/msks/_TR_1920_sub_05/gunzipmask/mapflow/_gunzipmask3/rCN_trans.nii.gz CN_RMS_CNR_0003.nii.gz
200914-19:32:05,827 nipype.workflow INFO:
	 [Node] Finished "_mask-RMS23".
200914-19:32:05,829 nipype.workflow INFO:
	 [Node] Setting-up "_mask-RMS24" in "/tmp/tmp0qoemq2m/msks/_TR_1920_sub_05/mask-RMS/mapflow/_mask-RMS24".
200914-19:32:05,833 nipype.workflow INFO:
	 [Node] Running "_mask-RMS24" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STR

200914-19:32:08,341 nipype.workflow INFO:
	 [Node] Finished "_mask-RMS36".
200914-19:32:08,343 nipype.workflow INFO:
	 [Node] Setting-up "_mask-RMS37" in "/tmp/tmp0qoemq2m/msks/_TR_1920_sub_05/mask-RMS/mapflow/_mask-RMS37".
200914-19:32:08,348 nipype.workflow INFO:
	 [Node] Running "_mask-RMS37" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/RMS_CNR_0003.nii.gz -mas /tmp/tmp2arwl9cp/msks/_TR_1920_sub_05/gunzipmask/mapflow/_gunzipmask5/rGPi_trans.nii.gz GPi_RMS_CNR_0003.nii.gz
200914-19:32:08,517 nipype.workflow INFO:
	 [Node] Finished "_mask-RMS37".
200914-19:32:08,519 nipype.workflow INFO:
	 [Node] Setting-up "_mask-RMS38" in "/tmp/tmp0qoemq2m/msks/_TR_1920_sub_05/mask-RMS/mapflow/_mask-RMS38".
200914-19:32:08,523 nipype.workflow INFO:
	 [Node] Running "_mask-RMS38" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/S

...

200914-19:46:42,428 nipype.workflow INFO:
	 [Node] Finished "_mask-CNR60".
200914-19:46:42,431 nipype.workflow INFO:
	 [Node] Setting-up "_mask-CNR61" in "/tmp/tmpn9dw13pj/msks/_TR_700_sub_01/mask-CNR/mapflow/_mask-CNR61".
200914-19:46:42,435 nipype.workflow INFO:
	 [Node] Running "_mask-CNR61" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM/CNR_0007.nii.gz -mas /tmp/tmp5vl4cnoz/msks/_TR_700_sub_01/gunzipmask/mapflow/_gunzipmask8/rVS_trans.nii.gz VS_CNR_0007.nii.gz
200914-19:46:42,601 nipype.workflow INFO:
	 [Node] Finished "_mask-CNR61".
200914-19:46:42,603 nipype.workflow INFO:
	 [Node] Setting-up "_mask-CNR62" in "/tmp/tmpn9dw13pj/msks/_TR_700_sub_01/mask-CNR/mapflow/_mask-CNR62".
200914-19:46:42,608 nipype.workflow INFO:
	 [Node] Running "_mask-CNR62" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivat

200914-19:46:42,948 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR17/LOC_CNR_0005.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR///LOC_CNR_0005.nii.gz
200914-19:46:42,949 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR18/LOC_CNR_0006.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR///LOC_CNR_0006.nii.gz
200914-19:46:42,951 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR19/LOC_CNR_0007.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR///LOC_CNR_0007.nii.gz
200914-19:46:42,953 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR20/LOC_CNR_0009.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-0

200914-19:46:43,6 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR49/STN_CNR_0001.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR///STN_CNR_0001.nii.gz
200914-19:46:43,8 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR50/STN_CNR_0002.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR///STN_CNR_0002.nii.gz
200914-19:46:43,10 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR51/STN_CNR_0003.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR///STN_CNR_0003.nii.gz
200914-19:46:43,12 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/_TR_700_sub_01/_mask-CNR52/STN_CNR_0005.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR70

200914-19:46:43,61 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS18/LOC_RMS_CNR_0006.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS///LOC_RMS_CNR_0006.nii.gz
200914-19:46:43,62 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS19/LOC_RMS_CNR_0007.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS///LOC_RMS_CNR_0007.nii.gz
200914-19:46:43,64 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS20/LOC_RMS_CNR_0009.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS///LOC_RMS_CNR_0009.nii.gz
200914-19:46:43,66 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS21/CN_RMS_CNR_0001.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP

200914-19:46:43,110 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS49/STN_RMS_CNR_0001.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS///STN_RMS_CNR_0001.nii.gz
200914-19:46:43,111 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS50/STN_RMS_CNR_0002.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS///STN_RMS_CNR_0002.nii.gz
200914-19:46:43,113 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS51/STN_RMS_CNR_0003.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS///STN_RMS_CNR_0003.nii.gz
200914-19:46:43,114 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/RMS/_TR_700_sub_01/_mask-RMS52/STN_RMS_CNR_0005.nii.gz -> /scratch/qbi/uqkgarn1/S